In [76]:
using LinearAlgebra

# task 1

In [77]:
#! Для написания документации стоило ознакомиться с https://docs.julialang.org/en/v1/manual/documentation/

    """
    Функция решает уравнение вида Lx = b. \n
    Она принимает на вход матрицу L::AbstractMatrix и правую часть b::AbstractVector. \n
    Далее происходит проверка размерности и принадлежности матрицы L к нижнетреугольным матрицам. \n
    Далее в цикле происходит проверка матрицы на невырожденность и в этом же цикле происходит
    вычисление элемента вектора решения.
    """
function forwardsub(L::AbstractMatrix, b::AbstractVector)
    #! В julia док-строка приписывается к объекту сверху.
    
    """
    Функция решает уравнение вида Lx = b. \n
    Она принимает на вход матрицу L::AbstractMatrix и правую часть b::AbstractVector. \n
    Далее происходит проверка размерности и принадлежности матрицы L к нижнетреугольным матрицам. \n
    Далее в цикле происходит проверка матрицы на невырожденность и в этом же цикле происходит
    вычисление элемента вектора решения.
    """
    
    #! size(b, 1) != size(L, 1) || ...
    if size(b)[1] != size(L)[1] || size(b)[1] != size(L)[2]
        return "Size of given system is incorrect"  #! throw(DimensionMismatch("..."))

    #! LinearAlgebra.istril(L)
    elseif L != LowerTriangular(L)
        return "L matrix is not lower triangular"  #! throw(...)
    else
        x = zeros(Float64, size(b))
        
        #! DRY: ниже есть код forwardsub!(x, L, b)
        #! return forwardsub!(x, L, b)
        for i in 1:(size(b)[1])
            if L[i,i] == 0
                return "det(L) = 0"  #! throw(LinearAlgebra.SingularException("..."))
            end
            x[i] = (b[i] - sum(x.*L[i,:]))/L[i,i]
        end
        return x
    end
    
    #! Плохо со стабильностью типов.
    #! Если операцию (решение системы) нельзя выполнить, то лучше выбрасывать ошибку об этом, а не возвращать строку.
    
    #! Используйте guard-if для меньшей вложенности кода (и, как следствие, лучшей читаемости).
    #! Т.е.
    #! size(b, 1) != size(L, 1) || size(b, 1) != size(L, 2) && throw(DimensionMismatch("..."))  # или через if
    #! !LinearAlgebra.istril(L) && throw(...)  # или через if
    #! Основной случай -- алгоритм подстановки
end

function forwardsub!(x::AbstractVector, L::AbstractMatrix, b::AbstractVector)
    if size(b)[1] != size(L)[1] || size(b)[1] != size(L)[2]
        return "Size of given system is incorrect"
    elseif L != LowerTriangular(L)
        return "L matrix is not lower triangular"
    else
#         x = zeros(size(b))

        #! Если подадут ненулевой x, код не заработает
        fill!(x, zero(eltype(x)))  #!
        
        for i in 1:(size(b)[1])
            if L[i,i] == 0
                return "det(L) = 0"
            end
            x[i] = (b[i] - sum(x.*L[i,:]))/L[i,i]
        end
        return x
    end
end

    """
    Функция решает уравнение вида Lx = b. \n
    Она принимает на вход матрицу L::AbstractMatrix и правую часть b::AbstractVector. \n
    Далее происходит проверка размерности и принадлежности матрицы L к нижнетреугольным матрицам. \n
    Далее в цикле происходит проверка матрицы на невырожденность и в этом же цикле происходит
    вычисление элемента вектора решения
    """
function backwardsub(U::AbstractMatrix, b::AbstractVector)
    if size(b)[1] != size(U)[1] || size(b)[1] != size(U)[2]
        return "Size of given system is incorrect"
    elseif U != UpperTriangular(U)
        return "U matrix is not lower triangular"
    else
        x = zeros(Float64, size(b))
        
        #! for in size(b, 1):-1:1
        for i in collect(size(b)[1]:-1:1)
            if U[i,i] == 0
                return "det(U) = 0"
            end
            x[i] = (b[i] - sum(x.*U[i, :]))/U[i,i]
        end
        return x
    end
    
    #! В целом те же замечания, что и для forwardsub
end

function backwardsub!(x::AbstractVector, U::AbstractMatrix, b::AbstractVector)
    if size(b)[1] != size(U)[1] || size(b)[1] != size(U)[2]
        return "Size of given system is incorrect"
    elseif U != UpperTriangular(U)
        return "U matrix is not lower triangular"
    else
#         x = zeros(size(b))
        
        #! Если подадут ненулевой x, код не заработает
        fill!(x, zero(eltype(x)))  #!
        
        for i in collect(size(b)[1]:-1:1)
            if U[i,i] == 0
                return "det(U) = 0"
            end
            x[i] = (b[i] - sum(x.*U[i,:]))/U[i,i]
        end
        return x
    end
end

backwardsub! (generic function with 1 method)

In [78]:
x = [0., 0., 0.]
L = [2 0 0; 1 1 0; 0 0 1]
U = [1 2 3; 0 5 6; 0 0 9]
b = [1, 2, 3]

3-element Vector{Int64}:
 1
 2
 3

In [79]:
print(forwardsub(L, b))

[0.5, 1.5, 3.0]

In [80]:
print(backwardsub(U, b))

[0.0, 0.0, 0.3333333333333333]

In [81]:
#!
@show L * forwardsub(L, b) - b;
@show U * backwardsub(U, b) - b;

L * forwardsub(L, b) - b = [0.0, 0.0, 0.0]
U * backwardsub(U, b) - b = [0.0, 0.0, 0.0]


In [82]:
?backwardsub

search: backwardsub backwardsub!



Функция решает уравнение вида Lx = b. 

Она принимает на вход матрицу L::AbstractMatrix и правую часть b::AbstractVector. 

Далее происходит проверка размерности и принадлежности матрицы L к нижнетреугольным матрицам. 

Далее в цикле происходит проверка матрицы на невырожденность и в этом же цикле происходит вычисление элемента вектора решения


In [83]:
print(backwardsub!(x, U, b))

[0.0, 0.0, 0.3333333333333333]

In [84]:
#!
# x = [0., 0., 0.]
L = [2 0 0; 1 1 0; 0 0 1]
U = [1 2 3; 0 5 6; 0 0 9]
b = [1, 2, 3]

x = float.(similar(b))
x = forwardsub!(x, L, b)
@show L \ b - x
@show L * x - b

x = backwardsub!(x, U, b)
@show U \ b - x
@show U * x - b

L \ b - x = [0.0, 0.0, 0.0]
L * x - b = [0.0, 0.0, 0.0]
U \ b - x = [0.0, 0.0, 0.0]
U * x - b = [0.0, 0.0, 0.0]


3-element Vector{Float64}:
 0.0
 0.0
 0.0

# task 2

In [85]:
function tridiagsolve(a::AbstractVector, b::AbstractVector, c::AbstractVector, f::AbstractVector)
    #! a, b, c, f = float.(a), float.(b), float.(c), float.(f)
    а, b, c, f = Float64.(a), Float64.(b), Float64.(c), Float64.(f)
    for i in collect(2:size(b)[1])
        b[i] = b[i] - a[i-1]/b[i-1]*c[i-1]
        f[i] = f[i] - a[i-1]/b[i-1]*f[i-1]
    end
#     x = zeros(Float64, n)
#     x[n] = f[n] / b[n]
#     for i in range(n-1, 1, step=-1)
#         x[i] = f[i] - c[i] * x[i+1]
#     end
#     return x
    return backwardsub(Tridiagonal(zeros(size(b)[1]-1), b, c), f)
end

function tridiagsolve(A::Tridiagonal, f)
    
    #! d1 == A.dl, d2 == A.d, d3 == A.du
    d1, d2, d3 = diag(A, -1), diag(A), diag(A, 1)
    return tridiagsolve(d1, d2, d3, f)
end

tridiagsolve (generic function with 2 methods)

In [86]:
a = [1, 1, 1]
b = [2, 2, 2, 2]
c = [1, 1, 1]
f = [1, 2, 3, 4]
A = Tridiagonal(a, b, c)


4×4 Tridiagonal{Int64, Vector{Int64}}:
 2  1  ⋅  ⋅
 1  2  1  ⋅
 ⋅  1  2  1
 ⋅  ⋅  1  2

In [87]:
print(tridiagsolve(a, b, c, f))

[0.0, 1.0, 0.0, 2.0]

In [88]:
print(tridiagsolve(A, f))

[0.0, 1.0, 0.0, 2.0]

In [89]:
print(f - A * tridiagsolve(A, f))

[0.0, 0.0, 0.0, 0.0]

Не реализована прогонка.
Решение через зануление диагональ и использование bacwardsub стоит квадратичное время.
Прогонка решает 3-диаг. систему за линейное.

# task 3

In [90]:
function solution_a()
    A = [8 9 4 -1;
         0 4 1 0;
         0 0 -1 6;
         0 0 0 11]
    b = [9, 3, -1, 2]
    x = backwardsub(A, b)
    return x, b - A * x
end

solution_a (generic function with 1 method)

In [91]:
print(solution_a())

([-0.15340909090909105, 0.2272727272727273, 2.090909090909091, 0.18181818181818182], [1.7763568394002505e-15, 0.0, 0.0, 0.0])

In [92]:
function solution_b()
    A = [-2 1 0 0 0;
         1 -2 1 0 0;
         0 1 -2 1 0;
         0 0 1 -2 1;
         0 0 0 1 -2]
    b = [1, 1, 1, 1, 1]
    x = tridiagsolve(Tridiagonal(A), b)
    return x, b - A * x
end

solution_b (generic function with 1 method)

In [93]:
print(solution_b())

([-2.5, -3.9999999999999996, -4.499999999999999, -4.0, -2.5], [-4.440892098500626e-16, 0.0, 1.7763568394002505e-15, -8.881784197001252e-16, 0.0])

In [94]:
function solution_c()
    A = [1 8 -3 9;
         0 4 10 -2;
         8 2 -5 1;
         3 1 6 12]
    b = [3, 6, 1, 4]
    F = lu(A)
    x = F \ b
    return x, b - A * x
end

solution_c (generic function with 1 method)

In [95]:
print(solution_c())

([0.2629310344827586, 0.4741379310344827, 0.41451149425287354, 0.02083333333333337], [0.0, 8.881784197001252e-16, 1.1102230246251565e-16, 0.0])